---
title: "Python標準ライブラリーでバッチ処理を行う方法"
description: "itertools.batched は、Python 3.12 で導入された「データを指定したサイズごとに分割して（バッチ化して）取り出す」ための非常に便利な関数です。これまで、機械学習のミニバッチ作成や、APIへの小分け送信、巨大なログの分割処理などを行う際は、トリッキーな自作関数やサードパーティ製ライブラリ（more-itertools など）を使う必要がありましたが、それが標準ライブラリだけでスマートに書けるようになりました。"
date: "2026-01-31"
categories: ["Python基礎", "標準ライブラリ"] 
jupyter: python3
---

In [ ]:
import pandas as pd
import duckdb
import faker

from itertools import batched

In [ ]:
data = ["A", "B", "C", "D", "E", "F", "G"]

# 3個ずつのバッチに分割
for batch in batched(data, 3):
    print(batch)

# 出力:
# ('A', 'B', 'C')
# ('D', 'E', 'F')
# ('G',)  <- 余った分も最後に出力される

In [ ]:
# fakerを使って100万行のダミーデータを生成し、DuckDBに挿入
fake = faker.Faker()

In [ ]:
%%time

data = [{
    "name":fake.name(),
    "sex":fake.random_element(elements=["M", "F"]),
    "age":fake.random_int(min=18, max=80),
    "address":fake.address().replace("\n", ", "),
    "phone_number":fake.phone_number(),
} for i in range(1_000_000)
]

df = pd.DataFrame(data)
# display(df.head())
# df.shape

In [ ]:
%%time

# ジェネレータ関数で無限にデータを生成できるようにしておく
def fake_data_generator():
    while True:
        yield {
            "name":fake.name(),
            "sex":fake.random_element(elements=["M", "F"]),
            "age":fake.random_int(min=18, max=80),
            "address":fake.address().replace("\n", ", "),
            "phone_number":fake.phone_number(),
        }

all_dfs = []
for batch in batched(fake_data_generator(), 10000): # 1万件ずつバッチ処理
    batch_df = pd.DataFrame(batch)
    all_dfs.append(batch_df)
    
    if len(all_dfs) >= 100: # 10000 * 100 = 1_000_000件でストップ
        break

df = pd.concat(all_dfs)

In [ ]:
output_csv = "fake_data_100k.csv"
df.to_csv(output_csv, index=False)

In [ ]:
%%timeit
reader = pd.read_csv(output_csv, chunksize=10000)

df_tmps = []
for df_chunk in reader:
    df_tmps.append(df_chunk)
df_tmp = pd.concat(df_tmps)

# df_tmp.head()

In [ ]:
%%timeit
df_tmp = pd.read_csv(output_csv)


In [ ]:
import time

user_ids = range(1000)
for batch in batched(user_ids, 50, strict=False):
    print(batch)
    time.sleep(1) # 負荷軽減のための待機